In [2]:
# Check the gene names
import pandas as pd
import requests

# Read Document 1 (assumes an Excel file with a column "Protein" containing gene names)
df1 = pd.read_excel('/Users/conny/Desktop/AlphaFold/Paper 1 data.xlsx', header=1)

# Now you can access the 'Protein' column
protein_column = df1['Protein'].dropna().unique()
print(protein_column)
gene_names = df1['Protein'].dropna().unique()  # unique gene names, drop NaNs if any


['ABCE1' 'ABCF1' 'ACAT1' 'ACLY' 'ACP1' 'AGPS' 'AHCY' 'AIM1' 'ALDH18A1'
 'ALYREF' 'ANKHD1' 'ANKRD17' 'AP2A1' 'AP2B1' 'AQR' 'ARHGEF2' 'ASCC3'
 'ATAD3A' 'ATP2A2' 'ATP5C1' 'ATP5L' 'ATXN2L' 'BCLAF1' 'C14orf166' 'CAD'
 'CAPRIN1' 'CARM1' 'CCAR2' 'CCNT1' 'CCT2' 'CCT3' 'CCT4' 'CCT5' 'CCT7'
 'CCT8' 'CDC5L' 'CDC73' 'CHD4' 'CHERP' 'CLINT1' 'CLTC' 'COPA' 'COPB1'
 'COPB2' 'CPSF7' 'CRYZ' 'CS' 'CSDE1' 'CSNK1A1' 'CSNK2A1' 'CSNK2A2' 'CTPS1'
 'DAP3' 'DAZAP1' 'DDX1' 'DDX17' 'DDX18' 'DDX23' 'DDX3X' 'DDX49' 'DDX5'
 'DDX58' 'DDX6' 'DECR1' 'DHX15' 'DHX29' 'DHX30' 'DHX36' 'DHX37' 'DHX38'
 'DHX9' 'DIMT1' 'DIS3' 'DIS3L' 'DKC1' 'DNAJA2' 'DNAJA3' 'EEF2' 'EIF2A'
 'EIF2AK2' 'EIF2C2' 'EIF3A' 'EIF3C' 'EIF3D' 'EIF3L' 'EIF3S9' 'EIF4A1'
 'EIF4G1' 'EIF4G2' 'EL52' 'EMG1' 'EWSR1' 'EXOSC10' 'EXOSC2' 'EXOSC3'
 'EXOSC4' 'EXOSC6' 'EXOSC9' 'FAM120A' 'FAM62A' 'FAM98A' 'FAM98B' 'FARSLA'
 'FASN' 'FBL' 'FLII' 'FRG1' 'FUBP1' 'FUS' 'FXR1' 'G3BP1' 'G3BP2' 'GAPDH'
 'GART' 'GFPT1' 'GNB2L1' 'GNL3' 'GOLGA2' 'HADHB' 'hCG_1994130'
 'hCG_2032

In [4]:
# Check the protein name
name_column = df1['Description'].dropna().unique()
print(name_column)

['ATP-binding cassette sub-family E member 1'
 'ATP-binding cassette sub-family F member 1'
 'Acetyl-CoA acetyltransferase, mitochondrial' 'ATP-citrate synthase'
 'Low molecular weight phosphotyrosine protein phosphatase'
 'Alkyldihydroxyacetonephosphate synthase, peroxisomal'
 'Adenosylhomocysteinase' 'Absent in melanoma 1 protein'
 'Delta-1-pyrroline-5-carboxylate synthase' 'THO complex subunit 4'
 'Ankyrin repeat and KH domain-containing protein 1'
 'Ankyrin repeat domain-containing protein 17'
 'AP-2 complex subunit alpha-1' 'AP-2 complex subunit beta'
 'Intron-binding protein aquarius'
 'Rho guanine nucleotide exchange factor 2'
 'Activating signal cointegrator 1 complex subunit 3'
 'ATPase family AAA domain-containing protein 3A'
 'Sarcoplasmic/endoplasmic reticulum calcium ATPase 2'
 'ATP synthase subunit gamma' 'ATP synthase subunit g, mitochondrial'
 'Ataxin-2-like protein' 'Bcl-2-associated transcription factor 1'
 'UPF0568 protein C14orf166' 'CAD protein' 'Caprin-1'
 'Histon

In [21]:
import requests
import pandas as pd

# Example list of (gene, description) pairs
gene_desc_list = [
    ("ABCE1", "ATP-binding cassette sub-family E member 1"),
    ("ZNF598", "Zinc finger protein 598"),
    # Add more (gene, description) pairs as needed
]

organism = "9606"
results_dict = {}
rows = []
for gene, description in gene_desc_list:
    query = f'gene_exact:{gene} AND protein_name:"{description}" AND organism_id:{organism}'
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": query,
        "fields": "accession,gene_names,protein_name,organism_name,sequence",
        "format": "json",
        "size": 500
    }
    res = requests.get(url, params=params)
    data = res.json()
    accessions = [r.get("primaryAccession") for r in data.get("results", [])]
    rows.append({
        "gene": gene,
        "accessions": ",".join(accessions) if accessions else "NOT_FOUND",
        "count": len(accessions)
    })

df = pd.DataFrame(rows)
df.to_csv("gene_accessions.csv", index=False)
print("Saved to gene_accessions.csv")

Saved to gene_accessions.csv


In [23]:
import requests
import pandas as pd
from tqdm import tqdm

df1 = pd.read_excel('/Users/conny/Desktop/AlphaFold/Paper 1 data.xlsx', header=1)
df1 = df1.dropna(subset=['Protein', 'Description'])
gene_desc_list = list(zip(df1['Protein'], df1['Description']))

organism = "9606"
rows = []
failed = []

for gene, description in tqdm(gene_desc_list, desc="Querying UniProt"):
    query = f'gene_exact:{gene} AND protein_name:"{description}" AND organism_id:{organism}'
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": query,
        "fields": "accession,gene_names,protein_name,organism_name,sequence",
        "format": "json",
        "size": 500
    }
    try:
        res = requests.get(url, params=params, timeout=30)
        data = res.json()
        accessions = [r.get("primaryAccession") for r in data.get("results", [])]
        rows.append({
            "gene": gene,
            "description": description,
            "accessions": ",".join(accessions) if accessions else "NOT_FOUND",
            "count": len(accessions)
        })
        if not accessions:
            failed.append((gene, description))
    except Exception as e:
        rows.append({
            "gene": gene,
            "description": description,
            "accessions": "ERROR",
            "count": 0
        })
        failed.append((gene, description))
        print(f"Error for {gene}: {e}")

df = pd.DataFrame(rows)
df.to_csv("gene_accessions.csv", index=False)
print("Saved to gene_accessions.csv")

if failed:
    print("\nFailed queries:")
    for gene, description in failed:
        print(f"{gene}: {description}")
else:
    print("All queries succeeded!")

Querying UniProt: 100%|██████████| 363/363 [02:44<00:00,  2.21it/s]

Saved to gene_accessions.csv

Failed queries:
AQR: Intron-binding protein aquarius
ATP5C1: ATP synthase subunit gamma
ATP5L: ATP synthase subunit g, mitochondrial
C14orf166: UPF0568 protein C14orf166
DDX58: Probable ATP-dependent RNA helicase DDX58
DECR1: 2,4-dienoyl-CoA reductase, mitochondrial
DHX15: Putative pre-mRNA-splicing factor ATP-dependent RNA helicase DHX15
DHX30: Putative ATP-dependent RNA helicase DHX30
DHX36: ATP-dependent RNA helicase DHX36
DIMT1: Probable dimethyladenosine transferase
DKC1: H/ACA ribonucleoprotein complex subunit 4
EL52: Heat shock protein HSP 90-alpha
hCG_2032701: Dolichyl-diphosphooligosaccharide--protein glycosyltransferase subunit STT3A
hCG_27698: Probable ATP-dependent RNA helicase DDX47
hCG_30204: Acetyl-CoA carboxylase 1
hCG_31253: Far upstream element-binding protein 3
KIF5B: Tyrosine-protein kinase receptor
MOV10: Putative helicase MOV-10
NOP2: Probable 28S rRNA (cytosine(4447)-C(5))-methyltransferase
NSUN5: Probable 28S rRNA (cytosine-C(5))-me

In [24]:
if failed:
    print("\nFailed queries:")
    for gene, description in failed:
        print(f"{gene}: {description}")
    # Save failed queries to a CSV file
    failed_df = pd.DataFrame(failed, columns=["gene", "description"])
    failed_df.to_csv("failed_gene_accessions.csv", index=False)
    print("Saved failed queries to failed_gene_accessions.csv")
else:
    print("All queries succeeded!")


Failed queries:
AQR: Intron-binding protein aquarius
ATP5C1: ATP synthase subunit gamma
ATP5L: ATP synthase subunit g, mitochondrial
C14orf166: UPF0568 protein C14orf166
DDX58: Probable ATP-dependent RNA helicase DDX58
DECR1: 2,4-dienoyl-CoA reductase, mitochondrial
DHX15: Putative pre-mRNA-splicing factor ATP-dependent RNA helicase DHX15
DHX30: Putative ATP-dependent RNA helicase DHX30
DHX36: ATP-dependent RNA helicase DHX36
DIMT1: Probable dimethyladenosine transferase
DKC1: H/ACA ribonucleoprotein complex subunit 4
EL52: Heat shock protein HSP 90-alpha
hCG_2032701: Dolichyl-diphosphooligosaccharide--protein glycosyltransferase subunit STT3A
hCG_27698: Probable ATP-dependent RNA helicase DDX47
hCG_30204: Acetyl-CoA carboxylase 1
hCG_31253: Far upstream element-binding protein 3
KIF5B: Tyrosine-protein kinase receptor
MOV10: Putative helicase MOV-10
NOP2: Probable 28S rRNA (cytosine(4447)-C(5))-methyltransferase
NSUN5: Probable 28S rRNA (cytosine-C(5))-methyltransferase
NUFIP2: Nucle

In [30]:
import pandas as pd
import requests

# Read the CSV with accessions
df = pd.read_csv("/Users/conny/Desktop/AlphaFold/src/notebooks/p1_gene_accessions.csv")

# Collect all unique accessions (skip NOT_FOUND and empty)
accession_set = set()
for acc_str in df['accessions']:
    if acc_str and acc_str != "NOT_FOUND":
        for acc in acc_str.split(","):
            acc = acc.strip()
            if acc:
                accession_set.add(acc)

print(accession_set)

{'Q14157', 'O43172', 'C9JHP4', 'A0A087WYY5', 'H0Y932', 'A0A5F9ZI66', 'Q14739', 'E5RJB9', 'F8W1S4', 'Q04637', 'A0A2R8Y4S2', 'Q15233', 'M0QY96', 'D6R991', 'D6RBK5', 'F8VQZ7', 'D6RAE9', 'D6REK4', 'Q9Y5B9', 'Q8WWM7', 'Q8NI61', 'P50991', 'P61247', 'O75569', 'Q8TDP1', 'H0YD77', 'P19105', 'K7ERG4', 'H0Y5H6', 'Q9Y4I1', 'B7Z4B8', 'A0A8V8TMA4', 'A0A2R8Y7H1', 'B5BUH5', 'Q9BRP8', 'A6PW58', 'P49790', 'A0A6Q8PH84', 'E9PM36', 'Q7Z2W4', 'Q5T7N0', 'Q9NQT5', 'K7ERM6', 'A0A2R8YDW2', 'A0A2R8Y6G9', 'E9PKT8', 'Q96EP5', 'Q13310', 'P62847', 'A2A3R5', 'A0A2R8Y7I0', 'A0A7I2V351', 'A0A2R8Y7M9', 'J3QLK9', 'A4D210', 'A0A2R8Y459', 'F8VTT8', 'B4DJV5', 'P25398', 'F8WD15', 'P62314', 'A0A7I2YQE1', 'R4GNJ2', 'H0YIC4', 'A0A2R8Y6L3', 'K7EM56', 'A0A286YFA2', 'O15226', 'V9HW96', 'A0A0A0MRL0', 'A0AAQ5BH61', 'Q15020', 'E9PI21', 'Q9BWF3', 'P36578', 'Q13610', 'C9JXK0', 'H0YD99', 'M0R2T0', 'Q9NYF8', 'P48634', 'B2R4W8', 'A0A3B3IRQ8', 'Q9NRX1', 'E9PDU5', 'H0YE22', 'Q9H9B4', 'K7EQA8', 'P52272', 'C9J8S3', 'Q86V81', 'Q9UKE5', 'Q8IWZ3

In [32]:
import requests
import sys
import json
import pandas as pd
from tqdm import tqdm  # <-- Add this import

# Extract the Protein ID from the column
protein_ids = accession_set

# API Parameters
params = {"fields": ["sequence"]}
headers = {"accept": "application/json"}

# Output file
output_file = "p1_protein_sequences.fasta"

def fetch_sequence(protein_id):
    url = f"https://rest.uniprot.org/uniprotkb/{protein_id}"
    response = requests.get(url, headers=headers, params=params)
    if response.ok:
        data = response.json()
        sequence_info = data.get("sequence", {})
        return sequence_info.get("value", None)
    else:
        print(f"Warning: Failed to fetch {protein_id}", file=sys.stderr)
        return None

results = {}
# Wrap tqdm around the outer loop
for entry in tqdm(protein_ids, desc="Fetching sequences"):
    protein_ids_split = entry.split(";")
    for protein_id in protein_ids_split:
        protein_id = protein_id.strip()
        if protein_id and protein_id not in results:
            sequence = fetch_sequence(protein_id)
            if sequence:
                results[protein_id] = sequence
            else:
                results[protein_id] = "Sequence Not Found"

with open(output_file, "w") as file:
    for protein_id, sequence in results.items():
        file.write(f">{protein_id}\n{sequence}\n")

print(f"Protein sequences saved to {output_file}")

Fetching sequences: 100%|██████████| 1106/1106 [07:35<00:00,  2.43it/s]

Protein sequences saved to p1_protein_sequences.fasta
